In [ ]:
import nltk 
from nltk import bigrams
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
data=pd.read_csv(r"C:\Users\pawan_300\Desktop\pos tagger\pos_tagger.csv")
train,test=train_test_split(data,train_size=0.8,test_size=0.2)

## Different words and tags

In [ ]:
words=[]
for i in train["Sentences"]:
    i=i.split(" ")
    for j in i:
        words.append(j)

In [ ]:
tags=[]
for i in train['Tags']:
    i=i.split(" ")
    for j in i:
        tags.append(j)

In [ ]:
def unigram(seq):
    temp={}
    for i in seq:
        if i in temp.keys():
            temp[i]+=1
        else:
            temp[i]=1
    return(temp)

In [ ]:
tags_unigram=unigram(tags)

## Prior probability

In [ ]:
def bigram(seq):
    temp={}
    bi=list(bigrams(seq))
    for i in bi:
        if i in temp.keys():
            temp[i]+=1
        else:
            temp[i]=1
    return(temp)

In [ ]:
transition_p=bigram(tags)

In [ ]:
for i in transition_p:
    transition_p[i]=transition_p[i]/tags_unigram[i[0]]

## Posterior probability

In [ ]:
def posterior(seq1,seq2):
    temp={}
    for i in zip(seq1,seq2):
        if i in temp.keys():
            temp[i]+=1
        else:
            temp[i]=1
    return(temp)

In [ ]:
emission_p=posterior(tags,words)

In [ ]:
for i in emission_p:
    emission_p[i]=emission_p[i]/tags_unigram[i[0]]

## Transition matrix

In [ ]:
tran_mat=np.zeros([len(set(tags)),len(set(tags))])

In [ ]:
temp_tag=list(set(tags))
for i in range(len(temp_tag)):
    for j in range(len(temp_tag)):
        if ((temp_tag[i],temp_tag[j]) in transition_p.keys()):
            tran_mat[i,j]=transition_p[temp_tag[i],temp_tag[j]]  

## Emission matrix

In [ ]:
emis_mat=np.zeros([len(set(tags)),len(set(words))])

In [ ]:
temp_word=list(set(words))
for i in range(len(temp_tag)):
    for j in range(len(temp_word)):
        if ((temp_tag[i],temp_word[j]) in emission_p.keys()):
            emis_mat[i,j]=emission_p[temp_tag[i],temp_word[j]] 

## Starting probability

In [ ]:
start_p=np.zeros(len(temp_tag))

In [ ]:
for i in train['Tags']:
    i=i.split(' ')
    for j in range(len(temp_tag)):
        if temp_tag[j]==i[0]:
             break
    start_p[j]+=1

In [ ]:
start_p=start_p/sum(start_p)

## Viterbi algorithm

In [ ]:
def viterbi2(obs,states,tran_mat,emis_mat,start_p):
    t1=np.zeros([len(states),len(obs)])
    t2=np.zeros([len(states),len(obs)])
    temp=np.zeros(len(states))
    temp1=np.zeros(len(states))
    for j in range(len(states)):
        if (states[j],obs[0]) in emis_mat.keys():
            t1[j,0]=start_p[j]*emis_mat[(states[j],obs[0])]
    for i in range(1,len(obs)):
        for j in range(len(states)):
            for k in range(len(states)):  
                if (states[k],states[j]) in tran_mat.keys() and (states[j],obs[i]) in emis_mat.keys():
                    temp[k]=t1[k,i-1]*tran_mat[(states[k],states[j])]*emis_mat[(states[j],obs[i])]
                    temp1[k]=t1[k,i-1]*tran_mat[(states[k],states[j])]
            t1[j,i]=max(temp)
            t2[j,i]=np.argmax(temp1)
    z=np.zeros(len(obs))
    for t in range(len(obs)):
        z[t]=np.argmax(t1[:,t])
    x=[]
    for i in range(t,-1,-1):
        z[i-1]=t2[int(z[i]),i]
        x.append(temp_tag[int(z[i-1])])
    return(x)

In [ ]:
x=[]
for i in range(train.shape[0]):
    x.append(viterbi2(train["Sentences"].iloc[i].split(" "),temp_tag,transition_p,emission_p,start_p))

In [ ]:
train.shape

(45872, 2)

In [ ]:
np.array(x).T

array([list(['LS', 'SYM', 'LS', 'SYM', '(', 'LS', 'SYM', 'SYM', 'SYM', 'SYM', 'SYM', 'LS', 'SYM', 'LS', 'SYM', '(', '(', 'EX', 'MD']),
       list(['SYM', 'SYM', 'SYM', 'SYM', 'SYM', 'LS', 'SYM', 'LS', 'SYM', 'LS', 'NNP', 'MD']),
       list(['(', 'SYM', 'SYM', 'NNPS', 'LS', 'SYM', 'SYM', 'LS', 'SYM', 'SYM', 'NNPS', 'LS', 'LS', 'SYM', 'SYM', 'LS', 'SYM', 'NNPS', 'LS', 'SYM', 'NNPS', 'LS', 'SYM', 'SYM', 'LS', 'LS', 'SYM', 'SYM', 'LS', 'SYM', 'SYM', 'LS', 'SYM', 'LS', 'SYM', 'SYM', 'NNP', 'MD']),
       ...,
       list(['SYM', 'SYM', 'LS', 'SYM', 'NNPS', 'LS', 'SYM', 'LS', 'SYM', 'LS', 'SYM', 'NN', 'DT', 'MD']),
       list(['MD', 'MD', 'SYM', 'SYM', 'NNPS', 'LS', 'LS', 'LS', 'SYM', 'PRP', 'WRB', 'SYM', 'SYM', 'LS', 'SYM', 'SYM', 'LS', 'SYM', 'EX', '$', 'SYM', 'SYM', 'LS', 'SYM', 'SYM', 'SYM', 'IN', 'MD']),
       list(['LS', 'LS', 'SYM', 'SYM', 'SYM', 'SYM', 'SYM', 'SYM', '$', 'SYM', 'SYM', 'LS', 'SYM', 'SYM', 'NNPS', 'LS', 'SYM', 'SYM', 'LS', 'SYM', '(', 'VBD', 'VBD', 'MD'])],
      d

In [ ]:
np.array(train["Sentences"].iloc[0].split(" ")).T

array(['In', 'a', 'sense', ',', 'the', 'showdown', 'promised', 'by',
       'Mr.', "Hammarskjold's", 'sudden', 'and', 'tragic', 'death', 'has',
       'been', 'avoided', ';', ';'], dtype='<U14')

In [ ]:
np.array(train["Tags"].iloc[0].split(" ")).T

array(['IN', 'DT', 'NN', ',', 'DT', 'NN', 'VBN', 'IN', 'NNP', 'NNP',
       'POS', 'JJ', 'CC', 'JJ', 'NN', 'VBZ', 'VBN', 'VBN', ':', ':'],
      dtype='<U3')

## Viterbi algorithm

In [ ]:
def viterbi(obs,states,tran_mat,emis_mat,start_p):
    t1=np.zeros([len(states),len(obs)])
    t2=np.zeros([len(states),len(obs)])
    temp=np.zeros(len(states))
    temp1=np.zeros(len(states))
    for j in range(len(states)):
        t1[j,0]=start_p[j]*emis_mat[j,[i for i in range(len(temp_word))if(obs[0]==temp_word[i])][0]]
    for i in range(1,len(obs)):
        for j in range(len(states)):
            for k in range(len(states)):
                temp[k]=t1[k,i-1]*tran_mat[k,j]*emis_mat[j,[ t for t in range(len(temp_word))if(obs[i]==temp_word[t])][0]]
                temp1[k]=t1[k,i-1]*tran_mat[k,j]
            t1[j,i]=max(temp)
            t2[j,i]=np.argmax(temp1)
    z=np.zeros(len(obs))
    for t in range(len(obs)):
        z[t]=np.argmax(t1[:,t])
    x=[]
    for i in range(t,2,-1):
        z[i-1]=t2[int(z[i]),i]
        x.append(temp_tag[int(z[i-1])])
    return(t1,t2,x)

In [ ]:
t1,t2,x=viterbi(train["Sentences"].iloc[0].split(" "),temp_tag,tran_mat,emis_mat,start_p)